In [1]:
from rdkit import Chem
from rdkit.Chem.SaltRemover import SaltRemover
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import Descriptors
import numpy as np
import pandas as pd
import networkx as nx
from rdkit.Chem.rdchem import BondType
from gap_deltaenergy import *

In [6]:
df_tox = pd.read_csv('data/toxic_small.csv')
df_exo = pd.read_csv('data/exo_small.csv')

In [7]:
#将外源性分子中有毒性分子的排除
tox_smi_list = set(df_tox['standard_smiles'].values)
exo_smi_list = set(df_exo['standard_smiles'].values)
print(len(tox_smi_list), len(exo_smi_list))
exo_smi_list = exo_smi_list - tox_smi_list
tox_smi_list = list(tox_smi_list)
exo_smi_list = list(exo_smi_list)
print(len(tox_smi_list), len(exo_smi_list))

3404 7519
3404 7235


In [4]:
tox_mols = [Chem.MolFromSmiles(smi) for smi in tox_smi_list]
exo_mols = [Chem.MolFromSmiles(smi) for smi in exo_smi_list]

[02:34:01] WARNING: not removing hydrogen atom without neighbors
[02:34:01] WARNING: not removing hydrogen atom without neighbors
[02:34:01] WARNING: not removing hydrogen atom without neighbors
[02:34:01] WARNING: not removing hydrogen atom without neighbors
[02:34:01] WARNING: not removing hydrogen atom without neighbors


In [12]:
tox_nbegapenergy1, tox_smiles_list1 =GetNBEandGapEnergy_asfeatures(tox_mols, tox_smi_list)
exo_nbegapenergy1, exo_smiles_list1 =GetNBEandGapEnergy_asfeatures(exo_mols, exo_smi_list)

In [14]:
col_list = ['sx_ave1', 'sx_ave2','sx_ave3','sx_ave4','sx_ave5',
            'sx_ave6',  'sx_ave7','sx_ave8','sx_ave9','sx_ave10',
            'sx_std1', 'sx_std2', 'sx_std3', 'sx_std4', 'sx_std5', 
            'sx_std6', 'sx_std7', 'sx_std8', 'sx_std9', 'sx_std10', 
            'nx_ave1', 'nx_ave2','nx_ave3','nx_ave4','nx_ave5',
            'nx_ave6',  'nx_ave7','nx_ave8','nx_ave9','nx_ave10',
            'nx_std1', 'nx_std2', 'nx_std3', 'nx_std4', 'nx_std5', 
            'nx_std6', 'nx_std7', 'nx_std8', 'nx_std9', 'nx_std10', 
            'oldnbe', 'oldstd', 'newnbe', 'newstd', 'max_gap']
df_tox_smi = pd.DataFrame({'standard_smiles':tox_smiles_list1})
df_tox_gapenergy = pd.DataFrame(tox_nbegapenergy1, columns= col_list)
df_tox = pd.concat([df_tox_smi, df_tox_gapenergy],axis=1)

df_exo_smi = pd.DataFrame({'standard_smiles':exo_smiles_list1})
df_exo_gapenergy = pd.DataFrame(exo_nbegapenergy1, columns= col_list)
df_exo = pd.concat([df_exo_smi, df_exo_gapenergy],axis=1)
df_exo.head()

,standard_smiles,sx_ave1,sx_ave2,sx_ave3,sx_ave4,sx_ave5,sx_ave6,sx_ave7,sx_ave8,sx_ave9,...,nx_std6,nx_std7,nx_std8,nx_std9,nx_std10,oldnbe,oldstd,newnbe,newstd,max_gap
0,CC1CCC(C(C)C2CCC3C4=CC(=O)C5CC(OC6OC(CO)C(O)C(...,42.691983,43.961735,49.334545,51.261818,44.888031,49.002041,42.448473,51.036170,48.417607,...,37.772912,36.843022,36.001760,35.452420,39.570915,63.598035,52.922845,23.099889,8.654226,25
1,C/C=C/S(=O)C(CC)SSCCC,72.830189,73.269841,101.574074,107.734694,83.327273,59.983051,49.875000,33.300000,26.400000,...,75.659057,74.887882,78.872284,76.015914,76.624479,48.497812,72.356102,23.556246,10.717787,10
2,COC(=O)C1OC(OC2CCC3(C)C(CCC4(C)C3CC=C3C5CC(C)(...,41.212329,48.534447,50.782904,50.498049,45.957276,51.478378,48.139565,48.682192,49.600852,...,45.842155,70.134041,74.861364,75.661650,71.309912,62.223076,57.948791,22.999512,8.658228,27
3,CCCCC/C=C\C/C=C\CCCCCCCCCC(=O)OC[C@H](COP(=O)(...,57.276018,46.964401,49.495017,55.969799,54.523649,53.853659,58.256140,56.304196,55.315412,...,32.687680,50.078332,52.101558,50.732123,58.659231,68.762972,64.378759,25.040933,8.536598,39
4,CS(=O)CCC(NC(=O)CCC(N)C(=O)O)C(=O)O,94.612903,104.720930,111.988764,109.862500,116.623188,106.609375,109.339286,99.480000,101.833333,...,120.775807,124.122119,131.255377,129.061685,125.552128,50.151560,112.883622,22.472381,9.064450,12


In [ ]:
#特征工程
# feature engineer
df_tox_gapenergy_fe = FE(df_tox)
df_exo_gapenergy_fe = FE(df_exo)


In [16]:
df_tox_all = descri(df_tox_gapenergy_fe)
df_exo_all = descri(df_exo_gapenergy_fe)
df_tox_all.insert(0,'label',np.ones(2560))
df_exo_all.insert(0,'label',np.zeros(7210))

[03:47:39] WARNING: not removing hydrogen atom without neighbors
[03:47:39] WARNING: not removing hydrogen atom without neighbors
[03:47:39] WARNING: not removing hydrogen atom without neighbors
[03:47:39] WARNING: not removing hydrogen atom without neighbors
[03:47:39] WARNING: not removing hydrogen atom without neighbors
[03:47:39] WARNING: not removing hydrogen atom without neighbors
[03:47:39] WARNING: not removing hydrogen atom without neighbors
[03:47:39] WARNING: not removing hydrogen atom without neighbors
[03:47:39] WARNING: not removing hydrogen atom without neighbors
/tmp/ipykernel_2838368/1470989083.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_tox_all.insert(0,'label',np.ones(2560))
/tmp/ipykernel_2838368/1470989083.py:17: Performance

In [17]:
df_tox_all.to_csv('data/tox_gapen_descri_sxnx.csv')
df_exo_all.to_csv('data/exo_gapen_descri_sxnx.csv')